# Long Puts OTM Strategy Eval

What if I continually buy puts that are way out of the money? Do you win big enough to outweigh the small losses?

Evaluation steps:

1. Pull stocks from Xano
2. Get historical prices on each stock
3. Get option chains on each stock (is it possible to get historical chains?)
4. Simulate the stock prices
5. Choose the stocks most likely to go down by expiration date
6. "Buy" puts and see

## Pull stocks from Xano

In [9]:
import requests
import pandas as pd
from datetime import datetime

In [6]:
url = 'https://iocalc.com/api:gPr_YnZX/getstocks'

response = requests.get(url)

stock_table = pd.json_normalize(response.json()['stocks'])

In [7]:
stock_table

,id,created_at,name,symbol,price,target,marketcap,week52high,week52low,dividendyield,...,pe,beta,change_1yr,change_ytd,pricetargethigh,pricetargetlow,symboltype,analysts,currency,totaltracking
0,54,1669442949620,Humana Inc.,HUM,540.95,485.87,48727349718,473.14,351.2,0.007368,...,18.229461173999,0.54182997393569,-0.058274921620204,-0.18272754710473,581.0,410.0,stock,23,USD,0
1,20,1669441631593,Meta Financial Group Inc.,CASH,44.66,72.50,1741265522,65.9,36.02,0.003454,...,6.2630944593554,1.1439973981583,0.48746215755319,-0.030673818303721,79.0,70.0,stock,4,USD,0
2,19,1669441649195,Global X Funds,QYLD,16.45,0.00,5871377700,22.43,19.11,0.121877,...,,,-0.0083631439208244,-0.079315006759802,0.0,0.0,,0,,0
3,11,1669441731840,Verizon Communications Inc,VZ,39.02,60.00,223153658344,58.44,49.09,0.058884,...,10.124939126336,0,-0.032254541085492,0.035421706459115,71.0,52.0,stock,26,USD,0
4,49,1669441937813,"Delta Air Lines, Inc.",DAL,35.10,51.56,24262923840,52.28,33.4,0.000000,...,-356.807703535,1.1989249496749,-0.05225,-0.029938587512794,67.0,40.0,stock,18,USD,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,74,1643141926509,"Oceaneering International, Inc.",OII,13.88,17.72,1282413359,18.2,8.27,0.000000,...,-36.130426529836,1.8863947677163,0.21916508538899,0.13616268788683,20.0,16.0,stock,9,USD,0
66,65,1643141629684,ALPS Fund Services,SBIO,33.95,0.00,146072500,64.04,31.96,0.000000,...,,,-0.38581129378127,-0.18631628787879,0.0,0.0,,0,,0
67,68,1656705855826,Rocket Lab USA Inc,RKLB,3.81,21.20,3917063728,21.34,8,0.000000,...,0,0,-0.16395014381592,-0.28990228013029,28.0,18.0,stock,5,USD,0
68,70,1669441644278,Kosmos Energy Ltd,KOS,6.41,5.96,1916009623,4.89,1.8,0.000000,...,-11.364502259128,2.3177199524754,0.58208955223881,0.22543352601156,8.0,4.1,stock,7,USD,0


## Get Historical Prices for Each Stock

In [8]:
config = pd.read_csv('../tradier_exploration/sandbox.config')
token = config.prod_token[0]

In [10]:
endpoint = 'https://api.tradier.com'
path = '/v1/markets/history'

all_stock_history = pd.DataFrame()

for symbol in stock_table['symbol']:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol':f'{symbol}', 
                'interval': 'daily', 
                'start': '2000-01-01', 
                'end': f'{datetime.today().date().isoformat()}'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )
    json_response = response.json()
    this_stock = pd.json_normalize(json_response['history']['day'])
    this_stock['symbol'] = symbol
    
    all_stock_history = pd.concat([all_stock_history, this_stock], axis = 0, ignore_index = True)

In [11]:
all_stock_history

,date,open,high,low,close,volume,symbol
0,2000-01-03,8.3125,8.375,7.375,7.5625,1287300,HUM
1,2000-01-04,7.375,7.875,7.375,7.6250,1238300,HUM
2,2000-01-05,7.5,7.875,7.5,7.8125,1096300,HUM
3,2000-01-06,7.75,8.25,7.5,8.1250,1026700,HUM
4,2000-01-07,8.1875,9.125,8.0625,8.7500,2419300,HUM
...,...,...,...,...,...,...,...
256723,2022-11-18,67.67,67.83,66.1392,67.4500,1529468,GFS
256724,2022-11-21,66.13,67.39,65.99,66.0700,1006054,GFS
256725,2022-11-22,66.19,68.125,65.43,67.3200,1698780,GFS
256726,2022-11-23,67.11,68.47,66.74,67.5800,1283507,GFS


## Get Option Chains

### Get Option Expirations

In [ ]:
path = '/v1/markets/options/expirations'

option_dates = pd.DataFrame()

for symbol in stock_table['symbol']:
    response = requests.get(f'{endpoint}{path}',
        params={'symbol': f'{symbol}', 
                'includeAllRoots': 'true'},
        headers={'Authorization': f'Bearer {token}', 
                 'Accept': 'application/json'}
    )
    
    these_dates = pd.DataFrame({'exp_dates': response.json()['expirations']['date'],
                                'symbol': symbol})
    option_dates = pd.concat([option_dates, these_dates], axis = 0, ignore_index = True)

In [21]:
option_dates

,exp_dates,symbol
0,2022-12-02,HUM
1,2022-12-09,HUM
2,2022-12-16,HUM
3,2022-12-23,HUM
4,2022-12-30,HUM
...,...,...
645,2023-04-21,GFS
646,2023-06-16,GFS
647,2023-07-21,GFS
648,2024-01-19,GFS


### Get Put Chains

In [26]:
path = '/v1/markets/options/chains'

all_puts = pd.DataFrame()

for symbol in stock_table['symbol']:
    dates = option_dates.loc[option_dates['symbol'] == symbol]
    for date in dates['exp_dates']:
        response = requests.get(f'{endpoint}{path}',
            params={'symbol': f'{symbol}', 
                    'expiration': f'{date}', 
                    'greeks': 'true'},
            headers={'Authorization': f'Bearer {token}', 
                     'Accept': 'application/json'}
        )

        json_response = response.json()

        tmp = pd.json_normalize(json_response['options']['option'])
        tmp = tmp.loc[tmp['option_type'] == 'put']

        all_puts = pd.concat([all_puts, tmp], axis = 0, ignore_index = True)

In [27]:
all_puts

,symbol,description,exch,type,last,change,volume,open,high,low,...,greeks.theta,greeks.vega,greeks.rho,greeks.phi,greeks.bid_iv,greeks.mid_iv,greeks.ask_iv,greeks.smv_vol,greeks.updated_at,greeks
0,HUM221202P00260000,HUM Dec 2 2022 $260.00 Put,Z,option,0.05,0.0,0,NaN,NaN,NaN,...,-0.025056,0.000020,0.050765,-0.105682,0.000000,1.818206,1.818206,0.347,2022-11-25 17:59:02,NaN
1,HUM221202P00280000,HUM Dec 2 2022 $280.00 Put,Z,option,0.01,-0.01,9,0.01,0.01,0.01,...,-0.027312,0.000020,0.054670,-0.105682,0.000000,1.939233,1.939233,0.347,2022-11-25 17:59:02,NaN
2,HUM221202P00290000,HUM Dec 2 2022 $290.00 Put,Z,option,0.4,0.0,0,NaN,NaN,NaN,...,-0.028440,0.000020,0.056622,-0.105682,0.000000,2.376545,2.376545,0.347,2022-11-25 17:59:02,NaN
3,HUM221202P00300000,HUM Dec 2 2022 $300.00 Put,Z,option,0.04,0.0,0,NaN,NaN,NaN,...,-0.029568,0.000020,0.058575,-0.105682,0.000000,1.937382,1.937382,0.347,2022-11-25 17:59:02,NaN
4,HUM221202P00310000,HUM Dec 2 2022 $310.00 Put,Z,option,NaN,NaN,0,NaN,NaN,NaN,...,-0.030696,0.000020,0.060527,-0.105682,0.000000,2.149952,2.149952,0.347,2022-11-25 17:59:02,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24762,GFS250117P00080000,GFS Jan 17 2025 $80.00 Put,Z,option,NaN,NaN,0,None,None,None,...,-0.013737,0.373598,0.474518,-0.827042,0.454364,0.518521,0.582679,0.517,2022-11-25 17:58:50,NaN
24763,GFS250117P00085000,GFS Jan 17 2025 $85.00 Put,Z,option,NaN,NaN,0,None,None,None,...,-0.013796,0.376661,0.458481,-0.780326,0.455109,0.513349,0.571589,0.512,2022-11-25 17:58:50,NaN
24764,GFS250117P00090000,GFS Jan 17 2025 $90.00 Put,Z,option,NaN,NaN,0,None,None,None,...,-0.013726,0.390308,0.441015,-0.734495,0.437923,0.502938,0.567953,0.506,2022-11-25 17:58:50,NaN
24765,GFS250117P00095000,GFS Jan 17 2025 $95.00 Put,Z,option,NaN,NaN,0,None,None,None,...,-0.013575,0.387897,0.422582,-0.690027,0.443254,0.502469,0.561685,0.501,2022-11-25 17:58:50,NaN


In [25]:
date

'exp_dates'